<a href="https://colab.research.google.com/github/JenBanks8585/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_rel
import seaborn as sns




In [0]:
 column_headers= ['party',
                   'handicapped-infants', 'water-project-cost-sharing',
                   'adoption-of-the-budget-resolution',
                   'physician-fee-freeze',
                   'el-salvador-aid',
                   'religious-groups-in-schools',
                   'anti-satellite-test-ban',
                   'aid-to-nicaraguan-contras',
                   'mx-missile',
                   'immigration',
                   'synfuels-corporation-cutback',
                   'education-spending',
                   'superfund-right-to-sue',
                   'crime','duty-free-exports',
                   'export-administration-act-south-africa']

In [3]:
# Reading the data from url
# Converting '?'s into NaN
# Adding column headers

df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None, names= column_headers, na_values= '?')
df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [38]:
#Revierwing column headers

df.columns


Index(['party', 'handicapped-infants', 'water-project-cost-sharing',
       'adoption-of-the-budget-resolution', 'physician-fee-freeze',
       'el-salvador-aid', 'religious-groups-in-schools',
       'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile',
       'immigration', 'synfuels-corporation-cutback', 'education-spending',
       'superfund-right-to-sue', 'crime', 'duty-free-exports',
       'export-administration-act-south-africa'],
      dtype='object')

In [5]:
#Reviewing rows/index

df.index

RangeIndex(start=0, stop=435, step=1)

In [6]:
#Checking the total NaN values for all attributes

df.isnull().sum()

party                                       0
handicapped-infants                        12
water-project-cost-sharing                 48
adoption-of-the-budget-resolution          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64

In [7]:
#recode data as numeric

df=df.replace({'y':1, 'n':0})
df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [8]:
#reviewing the number of representatives in each party

df['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [9]:
#Creating a new dataframe for republicans
#Also checking the number of republican representatives

rep=df[df['party']=='republican']
len(rep)

168

In [10]:
#Creating a new dataframe for democrats
#Also checking the number of democtrar representatives

dem=df[df['party']=='democrat']
len(dem)

267

In [11]:
# Overview of means for all attributes

summary_means = pd.DataFrame({'Dems': dem.mean(),'Reps': rep.mean()})
summary_means['difference']= summary_means['Dems']-summary_means['Reps']
summary_means

,Dems,Reps,difference
handicapped-infants,0.604651,0.187879,0.416772
water-project-cost-sharing,0.502092,0.506757,-0.004665
adoption-of-the-budget-resolution,0.888462,0.134146,0.754315
physician-fee-freeze,0.054054,0.987879,-0.933825
el-salvador-aid,0.215686,0.951515,-0.735829
religious-groups-in-schools,0.476744,0.897590,-0.420846
anti-satellite-test-ban,0.772201,0.240741,0.531460
aid-to-nicaraguan-contras,0.828897,0.152866,0.676031
mx-missile,0.758065,0.115152,0.642913
immigration,0.471483,0.557576,-0.086093


##2. Using hypothesis testing, find an issue that democrats 
##support more than republicans with p < 0.01

####Null hyp: Democrats do not suppot `Duty-free-export` more than Republicans
####Alternative hyp: `Duty-free-export` bill is supported more by Democrats than  by republican, DemXbar-RepXbar>0

####Significance level, alpha =0.05




In [12]:
# Looking at the overview of means, I can check for `duty-free-exports` attribute
# Democrats seem to support this bill more than Republicans had
# Pulling out their respective means

dem_duty_free_mean=summary_means['Dems'].loc[summary_means.index=='duty-free-exports']
rep_duty_free_mean=summary_means['Reps'].loc[summary_means.index=='duty-free-exports']
print(dem_duty_free_mean)
print(rep_duty_free_mean)

duty-free-exports    0.63745
Name: Dems, dtype: float64
duty-free-exports    0.089744
Name: Reps, dtype: float64


In [30]:
#Performing a t-test 
result= ttest_ind(dem['duty-free-exports'],rep['duty-free-exports'], nan_policy='omit')
result.statistic
result.pvalue

5.997697174347365e-32

In [33]:
# Another way of pulling out the values
# Converting scientific notation into general format

print('p-value is', f'{result.pvalue:.50f}')

p-value is 0.00000000000000000000000000000005997697174347364554


Conclusion: Since pvalue < 0.05, or even with a more stringent threshold,
then we can reject the null hypothesis and accept the alternative hypothesis 
that the bill for `duty-free export` is supported more by Democrats


##3.Using hypothesis testing, find an issue that republicans 
##support more than democrats with p < 0.01

####Null hyp: Republicans do not support `physician-free-freeze` more than Dem
####Alternative hyp: `physician-free` bill is supported more by Republicans than  by Democrats, RepXbar-DemXbar>0

####Significance level, alpha =0.05


In [14]:
# Looking at the overview of means, I can check for `physician-fee-freeze` attribute
# Republicans seem to support this bill more than Democrats had
# Pulling out their respective means

dem_physician_mean=summary_means['Dems'].loc[summary_means.index=='physician-fee-freeze']
rep_physician_mean=summary_means['Reps'].loc[summary_means.index=='physician-fee-freeze']
print(dem_physician_mean)
print(rep_physician_mean)

physician-fee-freeze    0.054054
Name: Dems, dtype: float64
physician-fee-freeze    0.987879
Name: Reps, dtype: float64


In [15]:
# Looking at the overview of means, I can check for `physician-fee-freeze` attribute
# Republicans seem to support this bill more than Democrats had
# Pulling out their respective means

dem_physician_mean=summary_means['Dems'].loc[summary_means.index=='physician-fee-freeze']
rep_physician_mean=summary_means['Reps'].loc[summary_means.index=='physician-fee-freeze']
print(dem_physician_mean)
print(rep_physician_mean)

physician-fee-freeze    0.054054
Name: Dems, dtype: float64
physician-fee-freeze    0.987879
Name: Reps, dtype: float64


In [16]:
# Looking at the overview of means, I can check for `physician-fee-freeze` attribute
# Republicans seem to support this bill more than Democrats had
# Pulling out their respective means

dem_physician_mean=summary_means['Dems'].loc[summary_means.index=='physician-fee-freeze']
rep_physician_mean=summary_means['Reps'].loc[summary_means.index=='physician-fee-freeze']
print(dem_physician_mean)
print(rep_physician_mean)

physician-fee-freeze    0.054054
Name: Dems, dtype: float64
physician-fee-freeze    0.987879
Name: Reps, dtype: float64


In [17]:
# Looking at the overview of means, I can check for `physician-fee-freeze` attribute
# Republicans seem to support this bill more than Democrats had
# Pulling out their respective means

dem_physician_mean=summary_means['Dems'].loc[summary_means.index=='physician-fee-freeze']
rep_physician_mean=summary_means['Reps'].loc[summary_means.index=='physician-fee-freeze']
print(dem_physician_mean)
print(rep_physician_mean)

physician-fee-freeze    0.054054
Name: Dems, dtype: float64
physician-fee-freeze    0.987879
Name: Reps, dtype: float64


In [18]:
#Performing a t-test 
ttest_ind(rep['physician-fee-freeze'],dem['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

In [0]:
#notice that pvalue above is practically 0.


####Conclusion: With a pvalue that is practically 0, we can reject the null hypothesis, and accept the alternative hypothesis that Republicans 
####support `physician-fee-freeze` bill more than by Democrats.

##4. Using hypothesis testing, find an issue where the difference 
##between republicans and democrats has p > 0.1 
##(i.e. there may not be much of a difference)

####Null hyp: There is a significant difference between the support of the two parties on `water-project-cost-sharing` bill.
####Alternative hyp: In `water-project-cost-sharing`  bill,  there is no significant difference between Republican and Democratic support

####Significance level, alpha =0.05

In [22]:
# Looking at the overview of means, I can check for `water-project-cost-sharing` attribute
# has the least absolute differemce between the means
# Pulling out their respective means
dem_water_project_mean=summary_means['Dems'].loc[summary_means.index=='water-project-cost-sharing']
rep_water_project_mean=summary_means['Reps'].loc[summary_means.index=='water-project-cost-sharing']
print(dem_water_project_mean)
print(rep_water_project_mean)


water-project-cost-sharing    0.502092
Name: Dems, dtype: float64
water-project-cost-sharing    0.506757
Name: Reps, dtype: float64


In [23]:
#Performing a t-test 
ttest_ind(dem['water-project-cost-sharing'],rep['water-project-cost-sharing'], nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

####Conclusion: Since p-value is greater than the threshold set at 0.05, then we fail to reject the Null hypothesis.

In [24]:
#Another attribute with relative low difference is 'immigration'
#checking their respective means

dem_immigration_mean=summary_means['Dems'].loc[summary_means.index=='immigration']
rep_immigration_mean=summary_means['Reps'].loc[summary_means.index=='immigration']
print(dem_immigration_mean)
print(rep_immigration_mean)

immigration    0.471483
Name: Dems, dtype: float64
immigration    0.557576
Name: Reps, dtype: float64


In [25]:
#Performing a t-test 
ttest_ind(dem['immigration'],rep['immigration'], nan_policy='omit')

Ttest_indResult(statistic=-1.7359117329695164, pvalue=0.08330248490425066)

####Similar conclusion on immigration bill as the one on `water-project-cost-sharing`

#Stretch goals:

Refactor your code into functions so it's easy to rerun with arbitrary variables

In [0]:
#Performing a t-test 
#ttest_ind(dem['water-project-cost-sharing'],rep['water-project-cost-sharing'], nan_policy='omit')

In [43]:
#Creating a list of columns excluding `party`

new_df=list(df.columns.drop('party'))
new_df

['handicapped-infants',
 'water-project-cost-sharing',
 'adoption-of-the-budget-resolution',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups-in-schools',
 'anti-satellite-test-ban',
 'aid-to-nicaraguan-contras',
 'mx-missile',
 'immigration',
 'synfuels-corporation-cutback',
 'education-spending',
 'superfund-right-to-sue',
 'crime',
 'duty-free-exports',
 'export-administration-act-south-africa']

In [48]:
# Creating a for loop that goes through all the columns

new_df

for col in new_df:
  print( ttest_ind(dem[col],rep[col], nan_policy='omit'), 'result for ', col)



Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18) result for  handicapped-infants
Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485) result for  water-project-cost-sharing
Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77) result for  adoption-of-the-budget-resolution
Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177) result for  physician-fee-freeze
Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68) result for  el-salvador-aid
Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20) result for  religious-groups-in-schools
Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31) result for  anti-satellite-test-ban
Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54) result for  aid-to-nicaraguan-contras
Ttest_indResult(statistic=16.437503268542994, pvalue=5.03079265310811e-47) result for  mx-missile
Tte